In [1]:
import pandas as pd

finetuned_dirname = "40-epoch-bert-base-finetuned-twitter15-tf"

data = pd.read_csv("../../data/processed/twitter15-tf_dataset.csv", lineterminator="\n")
data = data[['tweet_text', 'tvt2', 'label']]
print(data.shape)
data.head()

(742, 3)


,tweet_text,tvt2,label
0,rip elly may clampett: so sad to learn #beverl...,training,True
1,just in: missing afghan soldiers found trying ...,training,True
2,#riphulkhogan my heart is ripping like your sh...,training,False
3,a chick-fil-a manager allegedly banned this hi...,validation,False
4,islamic tribunal using sharia law in texas has...,validation,False


In [2]:
combined_data = data

In [3]:
import torch

class CustomTextDataset(torch.utils.data.dataset.Dataset):

    def __init__(self, texts, labels):
        self.labels = labels
        self.texts = texts
        self.attention_mask = None
        self.input_ids = None
        self.token_type_ids = None

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {
            "text": self.texts[idx],
            "label": self.labels[idx],
            "attention_mask": self.attention_mask[idx] if self.attention_mask else None,
            "input_ids": self.input_ids[idx] if self.input_ids else None,
            "token_type_ids": self.token_type_ids[idx] if self.token_type_ids else None
        }
        return sample
    
    def tokenize(self, tokenizer):
        self.attention_mask = []
        self.input_ids = []
        self.token_type_ids = []

        for text in self.texts:
            token = tokenizer(text, padding="max_length", truncation=True)
            
            self.attention_mask.append(token['attention_mask'])
            self.input_ids.append(token['input_ids'])
            self.token_type_ids.append(token['token_type_ids'])

c:\Users\msi_b\anaconda3\envs\pytorch-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
labels = []

labels_str = combined_data['label'].unique().tolist()
for i, d in combined_data.iterrows():
    lab = labels_str.index(d['label'])
    labels.append(lab)
    
print(len(labels))
labels[:10]

742


[0, 0, 1, 1, 1, 1, 1, 1, 0, 1]

In [5]:
train_dataset = CustomTextDataset(
    [d['tweet_text'] for i, d in combined_data.iterrows() if d['tvt2'] == 'training'],
    [labels[i] for i, d in combined_data.iterrows() if d['tvt2'] == 'training'])
test_dataset = CustomTextDataset(
    [d['tweet_text'] for i, d in combined_data.iterrows() if d['tvt2'] == 'validation'],
    [labels[i] for i, d in combined_data.iterrows() if d['tvt2'] == 'validation'])
train_dataset[0]

{'text': 'rip elly may clampett: so sad to learn #beverlyhillbillies star donna douglas has passed away. URL\r',
 'label': 0,
 'attention_mask': None,
 'input_ids': None,
 'token_type_ids': None}

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [7]:
# inputs = tokenizer(["you're stuck in a timewrap from 2004 though", "summa lumma dumma lumma"], padding="max_length", truncation=True)
# for k,v in inputs.items():
#     print(k)

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset.tokenize(tokenizer)
test_dataset.tokenize(tokenizer)

In [9]:
print(len(train_dataset))
print(len(test_dataset))

497
152


### Fine Tuning

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",
                                                           output_hidden_states=False,
                                                           num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments

epochs = 40
batch_size = 8
save_steps = (round((len(train_dataset)/batch_size) + 0.5)) * epochs
# save_steps = 1_000_000

training_args = TrainingArguments(
    output_dir=f"../../data/models/{finetuned_dirname}",
    num_train_epochs=epochs,
    save_steps=save_steps,
    logging_steps=300,
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    logging_strategy="epoch"
)

print(f"Save Steps : {save_steps}")

Save Steps : 2520


In [12]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

C:\Users\msi_b\AppData\Local\Temp\ipykernel_12828\3043712826.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [14]:
import time

start = time.time()

trainer.train()

print(f"Execution Time : {round(time.time() - start)} seconds")

  2%|▎         | 63/2520 [00:19<12:13,  3.35it/s] 

{'loss': 0.6719, 'learning_rate': 9.75e-06, 'epoch': 1.0}


                                                 
  2%|▎         | 63/2520 [00:21<12:13,  3.35it/s]

{'eval_loss': 0.6210176348686218, 'eval_accuracy': 0.6907894736842105, 'eval_runtime': 1.7674, 'eval_samples_per_second': 86.001, 'eval_steps_per_second': 10.75, 'epoch': 1.0}


  5%|▌         | 126/2520 [00:40<11:56,  3.34it/s]

...

{'eval_loss': 0.9527353048324585, 'eval_accuracy': 0.875, 'eval_runtime': 1.7924, 'eval_samples_per_second': 84.803, 'eval_steps_per_second': 10.6, 'epoch': 39.0}


100%|██████████| 2520/2520 [13:43<00:00,  3.33it/s]

{'loss': 0.0001, 'learning_rate': 0.0, 'epoch': 40.0}


                                                   
100%|██████████| 2520/2520 [13:45<00:00,  3.05it/s]

{'eval_loss': 0.9529910683631897, 'eval_accuracy': 0.875, 'eval_runtime': 1.789, 'eval_samples_per_second': 84.961, 'eval_steps_per_second': 10.62, 'epoch': 40.0}
{'train_runtime': 825.3855, 'train_samples_per_second': 24.086, 'train_steps_per_second': 3.053, 'train_loss': 0.03839224751240441, 'epoch': 40.0}
Execution Time : 825 seconds


In [15]:
trainer.evaluate()

100%|██████████| 19/19 [00:01<00:00, 11.28it/s]


{'eval_loss': 0.9529910683631897,
 'eval_accuracy': 0.875,
 'eval_runtime': 1.7794,
 'eval_samples_per_second': 85.422,
 'eval_steps_per_second': 10.678,
 'epoch': 40.0}